In [1]:
import cdsapi
import os
# import ecmwf.data as ecdata
import pandas as pd
import numpy as np
import xarray as xr

from datetime import datetime 
import sqlite3
#import folium
import multiprocessing
#import matplotlib.pyplot as plt

In [ ]:
#https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels?tab=download#manage-licences
#CDSAPI : https://cds.climate.copernicus.eu/how-to-api

c = cdsapi.Client()
c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'variable': [
            '2m_temperature', 'total_precipitation',
            'surface_pressure', '10m_u_component_of_wind',
            '10m_v_component_of_wind', 'relative_humidity'
        ],
        'year': '2020',
        'month': '01',
        'day': '01',
        'time': '12:00',
        'grid': [0.25, 0.25]  # 0.25 x 0.25 degree grid resolution
    },
    'download_single_levels.nc'
)

In [ ]:
c.retrieve(
    'reanalysis-era5-pressure-levels',
    {
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'variable': [
            'temperature', 'u_component_of_wind',
            'v_component_of_wind', 'geopotential', 
            'specific_humidity'
        ],
        'pressure_level': ['1', '2', '3', '5', '7', '10', '20', '30', '50', '70', '100', '125', '150', '175', '200', '225', '250', '300', '350', '400', '450', '500', '550', '600', '650', '700', '750', '775', '800', '825', '850', '875', '900', '925', '950', '975', '1000'],
        'year': '2020',
        'month': '01',
        'day': '01',
        'time': '12:00',
        'grid': [0.25, 0.25]  # 0.25 x 0.25 degree grid resolution
    },
    'download_pressure_levels.nc'
)

In [13]:
import xarray as xr
import rioxarray

# Open the NetCDF file using xarray (not rioxarray.open_rasterio)
#data = xr.open_dataset('download_single_levels.nc')
data = xr.open_dataset('download_pressure_levels.nc')
# To add rioxarray's geospatial functionalities, convert the dataset after loading
data = data.rio.write_crs("epsg:4326", inplace=True)  # Assuming the data has a known CRS
# Selecting specific pressure levels
selected_levels = data.sel(pressure_level=[1])  # Adjust the name 'pressure_level' as needed
print(selected_levels)

print(selected_levels.sizes)
data.close()
# Optionally, if you need to access or use specific rioxarray functionalities:
# data.rio.your_method_here()


<xarray.Dataset> Size: 21MB
Dimensions:         (valid_time: 1, pressure_level: 1, latitude: 721,
                     longitude: 1440)
Coordinates:
    number          int64 8B ...
  * valid_time      (valid_time) datetime64[ns] 8B 2020-01-01T12:00:00
  * pressure_level  (pressure_level) float64 8B 1.0
  * latitude        (latitude) float64 6kB 90.0 89.75 89.5 ... -89.75 -90.0
  * longitude       (longitude) float64 12kB 0.0 0.25 0.5 ... 359.2 359.5 359.8
    expver          <U4 16B ...
    spatial_ref     int64 8B 0
Data variables:
    t               (valid_time, pressure_level, latitude, longitude) float32 4MB ...
    u               (valid_time, pressure_level, latitude, longitude) float32 4MB ...
    v               (valid_time, pressure_level, latitude, longitude) float32 4MB ...
    z               (valid_time, pressure_level, latitude, longitude) float32 4MB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 4MB ...
Attributes:
    GRIB_centre:  